# Описание задачи
## В задачи пректа входит:
- Нахождение данных для обученяи модели: парсинг и/или иные источники;
- Предобработка полученных данных
- Создание новых признаков с использованием полученных данных;
- Отбор признаков;
- Отбор моделей;
- Обучение и тестирование отобранных моделей на отобранных признаках;
- Подбор макропараметров модели на отобранных признаках;
- Получение предсказанных моделью значений, подготовка и отправка submission

# Импорт библиотек, установка параметров, определение функций
## Импорт бибилиотек

In [8]:
import pandas as pd
import numpy as np
import json 

from pprint import pprint

import warnings
import os

## Установка параметров

In [9]:
%pylab inline

warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 70)    # выведем больше строк
pd.set_option('display.max_columns', 30) # выведем больше колонок

# альтернативные пути для kaggle и локальный (НЕ ЗАБЫВАТЬ МЕНЯТЬ!)
path = './Project_5_data/'
# path = '/kaggle/input/.....'

# фиксация randomstate
RANDOM_SEED = 42

Populating the interactive namespace from numpy and matplotlib


## Определение функций

In [10]:
def describe_df(df):
    '''
    Вывод простой статистки DataFrame
    '''
    desc = df.describe(include='all',percentiles=[0.5]).drop(index=['mean','std','50%']).T 
    desc['type'] = [type(x) for x in df.iloc[0]]
    desc['NaN'] = df.isna().sum()
    desc.unique = df.nunique()
    desc.top = df.mode(axis=0).iloc[0]
    desc.freq = [df[col].value_counts().iloc[0] for col in  df.columns]
    
    return desc

def cat_visualisation():
    pass

def num_visualisation():
    pass

# Изучение  данных для предсказания

Что бы понимать какие признаки нам нужны для обучения модели нам необходимо понять какие признаки описывают наблюдения, для которых мы должны предсказать целевую переменную.  

Поэтому для начала исследуем данные эти наблюдения описывающие.

In [11]:
df_test = pd.read_csv(f'{path}test.csv')

### Общий обзор

In [39]:
df_test.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Columns: 32 entries, bodyType to Таможня
dtypes: int64(6), object(26)
memory usage: 8.5+ MB


In [40]:
describe_df(df_test)

,count,unique,top,freq,min,max,type,NaN
bodyType,34686,24,седан,13147,NaN,NaN,<class 'str'>,0
brand,34686,12,BMW,4473,NaN,NaN,<class 'str'>,0
car_url,34686,34686,https://auto.ru/cars/used/sale/audi/100/101694...,1,NaN,NaN,<class 'str'>,0
color,34686,16,чёрный,11010,NaN,NaN,<class 'str'>,0
complectation_dict,6418,2364,"{""id"":""4562904"",""name"":""Elegance"",""available_o...",51,NaN,NaN,<class 'float'>,28268
description,34686,31732,Выгода до 82 000 руб. при обмене на Ваш автомо...,264,NaN,NaN,<class 'str'>,0
engineDisplacement,34686,55,2.0 LTR,7391,NaN,NaN,<class 'str'>,0
enginePower,34686,315,249 N12,1708,NaN,NaN,<class 'str'>,0
equipment_dict,24690,23705,"{""leather"":true}",108,NaN,NaN,<class 'str'>,9996
fuelType,34686,5,бензин,28601,NaN,NaN,<class 'str'>,0


Всего в исходном валидационном датасете 32 колонки с разными типами данных (6 int, 26 object) и разыми типами признаков:   
-бинарные (в колонках `vendor`, `ПТС`, `Руль`);   
-категориальные (например в колонках `bodyType`, `brand`, `color`, `Владельцы`);  
-количественные (например в `enginePower` или `mileage`).  

Некоторын колонки содержат словари или длинные строки, содержащие сразу несколько признаков. 

Колонка `brand` содержит 12 уникальных значений. Это позволяет нам существенно сократить обьем работы по парсингу, ведь релевантными будут только данные по этим 12 брендам, поэтому только по ним и будем собирать информацию.

Колонка `car_url` содержит ссылки на изображения на сайте ***auto.ru***, что как бы намекает, что валидационный набор спарсен с сайта ***auto.ru***. Воспользуемся этим же источником.

Три колонки  `priceCurrency`, `Состояние` и `Таможня` содержат только одно уникальное хначение, они ненесут никакой информации, поэтому исключим их из рассмотрения.  
  
Теперь следует ознакомиться с  каждым признак/колонку более детально. Это позволит нам понять в каком виде/формате содержится информация в колонках и соответсвенно сформировать набор данных для обучения в том же виде/формате. Так же детальное ознакомлене позволит окончательно определится с необходимость включения колонок в рассмотрение моделью.

### Детальное ознакомление
#### bodyType

In [21]:
df_test.bodyType.unique()

array(['лифтбек', 'внедорожник 5 дв.', 'хэтчбек 5 дв.', 'седан',
       'компактвэн', 'универсал 5 дв.', 'пикап одинарная кабина',
       'хэтчбек 3 дв.', 'купе', 'кабриолет', 'минивэн',
       'пикап двойная кабина', 'внедорожник 3 дв.', 'родстер', 'микровэн',
       'седан 2 дв.', 'купе-хардтоп', 'фастбек', 'тарга',
       'внедорожник открытый', 'лимузин', 'пикап полуторная кабина',
       'седан-хардтоп', 'фургон'], dtype=object)

#### brand

In [16]:
df_test.brand.unique()

array(['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI'],
      dtype=object)

#### car_url

In [23]:
df_test.car_url.sample(5)

5098     https://auto.ru/cars/used/sale/audi/a6/1101047...
20445    https://auto.ru/cars/used/sale/toyota/land_cru...
4091     https://auto.ru/cars/used/sale/audi/a6/1099444...
8878     https://auto.ru/cars/used/sale/bmw/x1/11007758...
3547     https://auto.ru/cars/used/sale/audi/a3/1101218...
Name: car_url, dtype: object

в этой колонке содержаться ссылки на обьявления о продаже, послужившие источником при парсинге `test`. В настоящий момент ссылки уже не активны, потому колокна будет исключена из рассмотрения.

### color

In [24]:
df_test.color.unique()

array(['синий', 'чёрный', 'серый', 'коричневый', 'белый', 'пурпурный',
       'бежевый', 'серебристый', 'красный', 'зелёный', 'жёлтый',
       'голубой', 'оранжевый', 'фиолетовый', 'золотистый', 'розовый'],
      dtype=object)

### complectation_dict

In [25]:
df_test.complectation_dict[2]

'{"id":"20026336","name":"Ambition","available_options":["heated-wash-system","airbag-passenger","lock","door-sill-panel","electro-mirrors","mirrors-heat","cooling-box","computer","seat-transformation","wheel-power","fabric-seats","airbag-side","abs","wheel-leather","climate-control-1","esp","adaptive-light","audiopreparation","ashtray-and-cigarette-lighter","front-centre-armrest","electro-window-back","16-inch-wheels","body-mouldings","condition","airbag-driver","isofix","aux","electro-window-front","light-sensor","hcc","ptf","rain-sensor","tyre-pressure","audiosystem-cd","front-seats-heat","wheel-configuration2","wheel-configuration1","immo","12v-socket","third-rear-headrest"]}'

### description

In [32]:
print(df_test.description[0])

Все автомобили, представленные в продаже, проходят тщательную проверку по более 40 параметрам. Предоставляем гарантию юридической чистоты, а так же год технической гарантии на двигатель и КПП. Бесплатный тест-драйв. Возможно оформление автомобиля в кредит!

Преимущества автокредитования:
— Первоначальный взнос от 0%;
— Более 30 кредитных программ;
— Процентная ставка от 6% годовых;
— Срок кредита от 6 месяцев до 7 лет;
— Оформление кредита по двум документам;
— Досрочное погашение без штрафов и комиссий;
— Сумма кредита до 2 млн рублей;
— Оформление КАСКО – по желанию;
— Без справок и поручителей.

Сотрудничаем с 12 аккредитованными и сертифицированными банками РФ, среднее время ожидания решения банка 20–30 минут.

При покупке автомобиля по программе «Trade-in – выгодный обмен» или в кредит, получите дополнительную скидку до 80 000 рублей на данный автомобиль!

Записаться на тест-драйв, а так же получить подробную информацию можно у специалистов автоцентра по указанному номеру телефона

В колонке содержится текст, относящийся скорее к продавцу, чем к конкретному автомобилю. Поэтому на данном этапе ( и вероятно вообще) колонка исключается из рассмотрения.

### engineDisplacement

In [33]:
df_test.engineDisplacement.unique()

array(['1.2 LTR', '1.6 LTR', '1.8 LTR', '2.0 LTR', '1.4 LTR', '1.3 LTR',
       '1.0 LTR', '3.6 LTR', '1.5 LTR', '1.9 LTR', '2.8 LTR', '1.1 LTR',
       '2.5 LTR', '4.2 LTR', '3.0 LTR', '4.0 LTR', '5.9 LTR', '2.7 LTR',
       '3.1 LTR', '2.4 LTR', '5.2 LTR', '3.2 LTR', '4.1 LTR', '6.3 LTR',
       '2.3 LTR', '6.0 LTR', '2.2 LTR', '3.7 LTR', '2.9 LTR', '5.0 LTR',
       '3.3 LTR', '2.1 LTR', '2.6 LTR', ' LTR', '3.5 LTR', '1.7 LTR',
       '0.7 LTR', '4.4 LTR', '4.8 LTR', '5.4 LTR', '6.6 LTR', '4.9 LTR',
       '3.8 LTR', '3.4 LTR', '3.9 LTR', '4.6 LTR', '5.6 LTR', '4.5 LTR',
       '5.5 LTR', '6.2 LTR', '4.7 LTR', '4.3 LTR', '5.8 LTR', '5.3 LTR',
       '5.7 LTR'], dtype=object)

### enginePower

In [36]:
df_test.enginePower.unique()[:30]

array(['105 N12', '110 N12', '152 N12', '200 N12', '102 N12', '150 N12',
       '90 N12', '180 N12', '220 N12', '122 N12', '70 N12', '140 N12',
       '125 N12', '54 N12', '86 N12', '75 N12', '64 N12', '95 N12',
       '260 N12', '170 N12', '80 N12', '68 N12', '160 N12', '115 N12',
       '280 N12', '53 N12', '60 N12', '143 N12', '42 N12', '101 N12'],
      dtype=object)

### equipment_dict

In [37]:
pprint(df_test.equipment_dict[2])

'{"cruise-control":true,"tinted-glass":true,"esp":true,"adaptive-light":true,"multi-wheel":true,"xenon":true,"heated-wash-system":true,"ashtray-and-cigarette-lighter":true,"airbag-passenger":true,"front-centre-armrest":true,"lock":true,"door-sill-panel":true,"electro-mirrors":true,"electro-window-back":true,"16-inch-wheels":true,"mirrors-heat":true,"body-mouldings":true,"dark-interior":true,"combo-interior":true,"rear-seats-heat":true,"cooling-box":true,"park-assist-r":true,"knee-airbag":true,"engine-proof":true,"airbag-driver":true,"aux":true,"isofix":true,"electro-window-front":true,"light-sensor":true,"hcc":true,"airbag-curtain":true,"computer":true,"seat-transformation":true,"wheel-power":true,"alarm":true,"light-cleaner":true,"alloy-wheel-disks":true,"ptf":true,"leather-gear-stick":true,"rain-sensor":true,"airbag-side":true,"tyre-pressure":true,"audiosystem-cd":true,"abs":true,"power-child-locks-rear-doors":true,"front-seats-heat":true,"wheel-leather":true,"wheel-configuration2":t

### fuelType

In [38]:
df_test.fuelType.unique()

array(['бензин', 'дизель', 'гибрид', 'электро', 'газ'], dtype=object)

### image

In [40]:
df_test.image.sample(5)

6335     https://avatars.mds.yandex.net/get-autoru-vos/...
28656    https://autoru.naydex.net/mcQh17801/f1b5751dCi...
29485    https://autoru.naydex.net/f1bm7A798/f15c727Yw/...
16350    https://autoru.naydex.net/ESooa9149/468b9eoMem...
18571    https://avatars.mds.yandex.net/get-autoru-vos/...
Name: image, dtype: object

колонка исключается из рассмотрения

### mileage

In [42]:
df_test.mileage.sample(5)

5033     165000
29122    250000
26830    200212
23469    155000
18738    227000
Name: mileage, dtype: int64

### modelDate

In [43]:
df_test.modelDate.unique()

array([2013, 2017, 2008, 2009, 2016, 2012, 2015, 2010, 2006, 2000, 2007,
       1994, 2004, 1999, 2005, 1976, 2001, 1969, 1996, 1998, 1989, 1934,
       2014, 2011, 2018, 1986, 1997, 1990, 2019, 2002, 1991, 1987, 1980,
       1982, 1938, 1988, 2003, 1983, 1978, 1979, 1984, 1992, 1995, 1993,
       1985, 1974, 1966, 1977, 1981, 1972, 1968, 1975, 1949, 1937, 1936,
       1973, 1959, 1958, 2020, 1965, 1971, 1904, 1963, 1955, 1951, 1960],
      dtype=int64)

### model_info

In [50]:
df_test.model_info.sample(5)

27393    {"code":"PAJERO","name":"Pajero","ru_name":"Па...
13523    {"code":"CEFIRO","name":"Cefiro","ru_name":"Це...
25854    {"code":"PASSAT","name":"Passat","ru_name":"Па...
20359    {"code":"COROLLA","name":"Corolla","ru_name":"...
17675    {"code":"W124","name":"W124","ru_name":"W124",...
Name: model_info, dtype: object

Колонка содержит наименование модели в трех вариантах. Учитывая наличие `model_name` колонка исключается из рассмотрения

### model_name

In [54]:
df_test.model_name.sample(5)

29721        5ER
7139         S60
11817       JUKE
714      OCTAVIA
1199       RAPID
Name: model_name, dtype: object

### name

In [57]:
df_test.name.sample(5)

19652        4.6 AT (309 л.с.) 4WD
6606             2.0 AT (245 л.с.)
11849        5.6 AT (405 л.с.) 4WD
2949         4.2 AT (450 л.с.) 4WD
18400    500 4.7 AT (435 л.с.) 4WD
Name: name, dtype: object

In [59]:
df_test.loc[18400]

bodyType                                                            седан
brand                                                            MERCEDES
car_url                 https://auto.ru/cars/used/sale/mercedes/s_klas...
color                                                          коричневый
complectation_dict      {"id":"7856162","name":"S 500 4MATIC","availab...
description             Внимание! Только для клиентов AVILON Автомобил...
engineDisplacement                                                4.7 LTR
enginePower                                                       435 N12
equipment_dict          {"cruise-control":true,"esp":true,"windcleaner...
fuelType                                                           бензин
image                   https://avatars.mds.yandex.net/get-autoru-vos/...
mileage                                                            222486
modelDate                                                            2009
model_info              {"code":"S_KLA

### numberOfDoors

In [ ]:
df_test.numberOfDoors

### parsing_unixtime

In [ ]:
df_test.parsing_unixtime

### priceCurrency

In [ ]:
df_test.priceCurrency

### productionDate

In [ ]:
df_test.productionDate

### sell_id

In [ ]:
df_test.sell_id

### super_gen

In [ ]:
df_test.super_gen

### vehicleConfiguration

In [ ]:
df_test.vehicleConfiguration

### vehicleTransmission

In [ ]:
df_test.vehicleTransmission

### vendor

In [47]:
df_test.vendor.unique()

array(['EUROPEAN', 'JAPANESE'], dtype=object)

In [48]:
df_test.vendor.value_counts()

EUROPEAN    20682
JAPANESE    14004
Name: vendor, dtype: int64

In [56]:
df_test.brand[df_test.vendor == 'JAPANESE'].sample(5)   

14866      INFINITI
27005    MITSUBISHI
27896    MITSUBISHI
28594    MITSUBISHI
5354          HONDA
Name: brand, dtype: object

In [55]:
df_test.brand[df_test.vendor == 'EUROPEAN'].sample(5)   

29265         BMW
3241         AUDI
16530    MERCEDES
4570         AUDI
3681         AUDI
Name: brand, dtype: object

### Владельцы

In [ ]:
df_test.Владельцы

### Владение

In [31]:
df_test.Владение.head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Владение, dtype: object

### ПТС

In [16]:
df_test.ПТС.unique()

array(['Оригинал', 'Дубликат', nan], dtype=object)

In [19]:
df_test.ПТС.value_counts()

Оригинал    30098
Дубликат     4587
Name: ПТС, dtype: int64

### Привод

In [ ]:
df_test.Привод

### Руль

In [ ]:
df_test.Руль

### Состояние

In [32]:
df_test.Состояние.head()

0    Не требует ремонта
1    Не требует ремонта
2    Не требует ремонта
3    Не требует ремонта
4    Не требует ремонта
Name: Состояние, dtype: object

### Таможня

In [ ]:
df_test.Таможня